# Python features for linear algebra

While Python has advanced libraries such as `numpy` for linear algebra and matrix manipulation, simple operations can also be implemented with just built-in features.

## Vectors

Python uses `list`s to represent collections of values. This forms a natural way to represent a vector.

Lists are represented with `[` square brackets `]`.

In [ ]:
v0 = [0, 0, 0]
v1 = [0, 1, 2]
v2 = [1, 2, 1]
v3 = [2, 7, 8]

print(v1)
print(v1[0])  # Lists are indexed starting from 0

We can define functions that operate on lists and/or values:

In [ ]:
def all_zero(V):
    # Returns True if all elements of Vector V (list) are 0.
    # Otherwise return False.
    for elem in V:
        if elem != 0:
            return False
    return True

print(all_zero(v0))
print(all_zero(v1))

In [ ]:
def elem_mul(V, n):
    # Performs element-wise multiplication of vector V by scalar value n.
    # Returns the resulting vector V'.
    V_ = []  # Empty list
    for elem in V:
        V_.append(elem * n)
    return V_

def elem_div(V, n):
    # Performs element-wise division of vector V by scalar value n.
    # Returns the resulting vector V'.
    V_ = []
    for elem in V:
        V_.append(elem / n)
    return V_

def row_add(V1, V2):
    # Adds two rows together element-wise.
    # V1 and V2 are assumed to have same number of elements.
    V_ = []
    for elem1, elem2 in zip(V1, V2):
        V_.append(elem1 + elem2)
    return V_

print(elem_mul(v1, 4))
print(elem_div(v2, 2))
print(row_add(v1, v2))

## Matrices

Python has no built-in type for n-dimensional matrices. Instead, they can be composed from lists.

For example, a 2D matrix can be represented as a list of rows, with each row represented by a list.

In [ ]:
A = [v1, v2, v3]

print(A)
print(A[0])  # Each element of the matrix list is a row, also represented by a list
print(A[0][2])  # Each value in the matrix is thus referenced by two indexes

Functions can also be defined to operate on lists of lists:

In [ ]:
def get_first_nonzero_index(V):
    # Returns the index of the first non-zero element in vector V (list).
    for i, elem in enumerate(V):
        if elem != 0:
            return i

def is_row_echelon(A):
    # Returns True if matrix A is in row echelon form,
    # Otherwise returns False
    nonzero_column_index = None
    for row in A:
        if nonzero_column_index is None:
            nonzero_column_index = get_first_nonzero_index(row)
        else:
            this_column_index = get_first_nonzero_index(row)
            if this_column_index <= nonzero_column_index:
                return False
            else:
                nonzero_column_index = this_column_index
        # continue to next row
    return True

The above features can be used to perform basic procedures in linear algebra, such as row echelon reduction:

In [ ]:
def find_pivot(A, start_row=0):
    # Find the pivot, the first non-zero element in the leftmost column of matrix A.
    num_rows = len(A)
    num_cols = len(A[0])  # Assume all rows have same number of elements
    row = start_row
    col = 0
    
    while col < num_cols:
        if A[row][col] == 0:  # Not pivot
            # If last row reached, the whole column is 0s
            # Check the next column of numbers
            if row == num_rows - 1:  # last row
                col = col + 1
                row = start_row
            else:
                row = row + 1
        else:  # Pivot found
            return row, col
    # If this point is reached, no pivot was found
    raise ValueError("Pivot not found.")
    
    
def pivot(A, start_row=0):
    # Perform one iteration of pivoting the matrix A starting from a given row.
    # A is a list of rows (list).
    # Rows above and columns before p are assumed to be pivoted.
    # The matrix A wil be modified!
    num_rows = len(A)
    
    pivot_row, pivot_col = find_pivot(A, start_row)

    # Swap pivot row with start row
    A[start_row], A[pivot_row] = A[pivot_row], A[start_row]
    pivot_row = start_row

    # Normalise pivot row so that pivot is 1.
    # The pivot is assumed to be non-zero after row swap
    A[pivot_row] = elem_div(A[pivot_row], A[pivot_row][pivot_col])
    
    # Add multiples of the pivot row to each of the lower rows,
    # so every element in the pivot column of the lower rows equals 0.
    for row in range(pivot_row + 1, num_rows):  # p + 1 to last row
        # skip if pivot column value already 0
        if A[row][pivot_col] == 0:
            continue
        # Pivot is 1
        # Multiply pivot row by negative of pivot column value, add to row
        factor = A[row][pivot_col]
        A[row] = row_add(A[row],
                         elem_mul(A[pivot_row], -A[row][pivot_col]))

In [ ]:
def row_echelon(A):
    # Transform matrix A into row echelon form.
    for row in range(len(A)):
        if all_zero(A[row]):
            continue
        pivot(A, row)  # Skip above rows in each iteration

In [ ]:
from copy import deepcopy

A_ = deepcopy(A)  # Don't modify the original A; make a copy
print(A_)
row_echelon(A_)
print(A_)